## Tutorial 07 - Non linear Elliptic problem
**_Keywords: DEIM, POD-Galerkin_**

### 1. Introduction

In this tutorial, we consider a non linear elliptic problem in a two-dimensional spatial domain $\Omega=(0,1)^2$. We impose a homogeneous Dirichlet condition on the boundary $\partial\Omega$. The source term is characterized by the following expression
$$
g(\boldsymbol{x}; \boldsymbol{\mu}) = 100\sin(2\pi x_0)cos(2\pi x_1) \quad \forall \boldsymbol{x} = (x_0, x_1) \in \Omega.
$$

This problem is characterized by two parameters. The first parameter $\mu_0$ controls the strength of the sink term and the second parameter $\mu_1$ the strength of the nonlinearity. The range of the two parameters is the following:
$$
\mu_0,\mu_1\in[0.01,10.0]
$$
The parameter vector $\boldsymbol{\mu}$ is thus given by
$$
\boldsymbol{\mu} = (\mu_0,\mu_1)
$$
on the parameter domain
$$
\mathbb{P}=[0.01,10]^2.
$$


In order to obtain a faster approximation of the problem, we pursue a model reduction by means of a POD-Galerkin reduced order method. In order to preserve the affinity assumption the discrete empirical interpolation method will be used on the forcing term $g(\boldsymbol{x}; \boldsymbol{\mu})$.



### 2. Parametrized formulation

Let $u(\boldsymbol{\mu})$ be the solution in the domain $\Omega$.

The strong formulation of the parametrized problem is given by: for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})$ such that

$$ -\nabla^2u(\boldsymbol{\mu})+\frac{\mu_0}{\mu_1}(\exp\{\mu_1u(\boldsymbol{\mu})\}-1)=g(\boldsymbol{x}; \boldsymbol{\mu})$$
<br>
    
The corresponding weak formulation reads: for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})\in\mathbb{V}$ such that

$$a\left(u(\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)+c\left(u(\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)=f(v;\boldsymbol{\mu})\quad \forall v\in\mathbb{V}$$

where

* the function space $\mathbb{V}$ is defined as
$$
\mathbb{V} = \{v\in H_1(\Omega) : v|_{\partial\Omega}=0\}
$$
* the parametrized bilinear form $a(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$a(u, v;\boldsymbol{\mu})=\int_{\Omega} \nabla u\cdot \nabla v \ d\boldsymbol{x},$$
* the parametrized bilinear form $c(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$c(u, v;\boldsymbol{\mu})=\mu_0\int_{\Omega} \frac{1}{\mu_1}\big(\exp\{\mu_1u\} - 1\big)v \ d\boldsymbol{x},$$
* the parametrized linear form $f(\cdot; \boldsymbol{\mu}): \mathbb{V} \to \mathbb{R}$ is defined by
$$f(v; \boldsymbol{\mu})= \int_{\Omega}g(\boldsymbol{x}; \boldsymbol{\mu})v \ d\boldsymbol{x}.$$

The output of interest $s(\boldsymbol{\mu})$ is given by
$$s(\boldsymbol{\mu}) = \int_{\Omega} v \ d\boldsymbol{x}$$
is computed for each $\boldsymbol{\mu}$.

In [1]:
import os
import sys
sys.path.append('../../')

from mlnics import NN, Losses, Normalization, RONNData, IO, Training, ErrorAnalysis
from dolfin import *
from rbnics import *
import torch
import numpy as np
import time

torch.manual_seed(0)
np.random.seed(0)

### 3. Affine Decomposition 

For this problem the affine decomposition is straightforward:
$$a(u,v;\boldsymbol{\mu})=\underbrace{1}_{\Theta^{a}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_0(u,v)},$$
$$c(u,v;\boldsymbol{\mu})=\underbrace{\mu_0}_{\Theta^{c}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega}\frac{1}{\mu_1}\big(\exp\{\mu_1u\} - 1\big)v \ d\boldsymbol{x}}_{c_0(u,v)},$$
$$f(v; \boldsymbol{\mu}) = \underbrace{100}_{\Theta^{f}_0(\boldsymbol{\mu})} \underbrace{\int_{\Omega}\sin(2\pi x_0)cos(2\pi x_1)v \ d\boldsymbol{x}}_{f_0(v)}.$$
We will implement the numerical discretization of the problem in the class
```
class NonlinearElliptic(NonlinearEllipticProblem):
```
by specifying the coefficients $\Theta^{a}_*(\boldsymbol{\mu})$, $\Theta^{c}_*(\boldsymbol{\mu})$ and $\Theta^{f}_*(\boldsymbol{\mu})$ in the method
```
    def compute_theta(self, term):
```
and the bilinear forms $a_*(u, v)$, $c_*(u, v)$ and linear forms $f_*(v)$ in
```
    def assemble_operator(self, term):
```

In [2]:
@DEIM("online", basis_generation="Greedy")
@ExactParametrizedFunctions("offline")
class NonlinearElliptic(NonlinearEllipticProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        NonlinearEllipticProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.du = TrialFunction(V)
        self.u = self._solution
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)
        # Store the forcing term expression
        self.f = Expression("sin(2*pi*x[0])*sin(2*pi*x[1])", element=self.V.ufl_element())
        # Customize nonlinear solver parameters
        self._nonlinear_solver_parameters.update({
            "linear_solver": "mumps",
            "maximum_iterations": 20,
            "report": True
        })

    # Return custom problem name
    def name(self):
        return "NonlinearEllipticDEIM"

    # Return theta multiplicative terms of the affine expansion of the problem.
    @compute_theta_for_derivatives
    def compute_theta(self, term):
        mu = self.mu
        if term == "a":
            theta_a0 = 1.
            return (theta_a0,)
        elif term == "c":
            theta_c0 = mu[0]
            return (theta_c0,)
        elif term == "f":
            theta_f0 = 100.
            return (theta_f0,)
        elif term == "s":
            theta_s0 = 1.0
            return (theta_s0,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    @assemble_operator_for_derivatives
    def assemble_operator(self, term):
        v = self.v
        dx = self.dx
        if term == "a":
            du = self.du
            a0 = inner(grad(du), grad(v)) * dx
            return (a0,)
        elif term == "c":
            u = self.u
            mu = self.mu
            c0 = (exp(mu[1] * u) - 1) / mu[1] * v * dx
            return (c0,)
        elif term == "f":
            f = self.f
            f0 = f * v * dx
            return (f0,)
        elif term == "s":
            s0 = v * dx
            return (s0,)
        elif term == "dirichlet_bc":
            bc0 = [DirichletBC(self.V, Constant(0.0), self.boundaries, 1)]
            return (bc0,)
        elif term == "inner_product":
            du = self.du
            x0 = inner(grad(du), grad(v)) * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")


# Customize the resulting reduced problem
@CustomizeReducedProblemFor(NonlinearEllipticProblem)
def CustomizeReducedNonlinearElliptic(ReducedNonlinearElliptic_Base):
    class ReducedNonlinearElliptic(ReducedNonlinearElliptic_Base):
        def __init__(self, truth_problem, **kwargs):
            ReducedNonlinearElliptic_Base.__init__(self, truth_problem, **kwargs)
            self._nonlinear_solver_parameters.update({
                "report": True,
                "line_search": "wolfe"
            })

    return ReducedNonlinearElliptic

## 4. Main program

### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [3]:
mesh = Mesh("data/square.xml")
subdomains = MeshFunction("size_t", mesh, "data/square_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/square_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P1)

In [4]:
V = FunctionSpace(mesh, "Lagrange", 1)

### 4.3. Allocate an object of the NonlinearElliptic class

In [5]:
problem = NonlinearElliptic(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(0.01, 10.0), (0.01, 10.0)]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with a POD-Galerkin method

In [6]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(20, DEIM=21)
reduction_method.set_tolerance(1e-8, DEIM=1e-5)

### 4.5. Perform the offline phase

#### 4.5.1 Fit Reduction Method

In [7]:
reduction_method.initialize_training_set(50, DEIM=60)
reduced_problem = reduction_method.offline()

=                     DEIM preprocessing phase begins for                      =
=   { v_0 * (-1 + exp(f_9 * f_14)) / f_14 } * dx(<Mesh #0>[everywhere], {}),   =
=                                    where                                     =
=                                 f_14 = mu_1                                  =
=f_9 = solution of NonlinearEllipticDEIM (exact problem decorator: False, component: (None,))=
=               with id 49fc90be369b6ead57691b159f98bb7ecf8d1031               =

:::::::::::::::::::::::::::::::::::: DEIM 0 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (6.781387202594339, 2.7073796521897266)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 3.041220e-01
  2 SNES Function norm 3.634108e-02
  3 SNES Function norm 5.645992e-04
  4 SNES Function norm 1.373336e-07
  5 SNES Function norm 9.051914e-15
PETSc SNES solver converged in 5 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::

  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.585483e-02
  2 SNES Function norm 3.535813e-05
  3 SNES Function norm 1.494355e-10
PETSc SNES solver converged in 3 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 14 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (6.605133639551924, 2.907875296032336)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 3.541945e-01
  2 SNES Function norm 5.009208e-02
  3 SNES Function norm 1.126394e-03
  4 SNES Function norm 5.768115e-07
  5 SNES Function norm 1.527327e-13
PETSc SNES solver converged in 5 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 15 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (6.183974135698427, 4.293399322448203)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.110032e+00
  2 SNES Function norm 3.195261e-01
  3 SNES Function norm 4.514126e

PETSc SNES solver converged in 11 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 28 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (8.075116397662857, 5.695316378759787)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.351215e+00
  2 SNES Function norm 1.071292e+00
  3 SNES Function norm 9.572627e-01
  4 SNES Function norm 2.497697e-01
  5 SNES Function norm 2.926566e-02
  6 SNES Function norm 4.911466e-04
  7 SNES Function norm 1.422777e-07
  8 SNES Function norm 1.265023e-14
PETSc SNES solver converged in 8 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 29 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (4.077761139287737, 0.7009782845968291)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 2.476348e-02
  2 SNES Function norm 3.369795e-05
  3 SNES Function norm 1.117297e-10
PETSc SNES solver converged in

  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 4.582215e-01
  2 SNES Function norm 7.983241e-02
  3 SNES Function norm 2.943975e-03
  4 SNES Function norm 4.128025e-06
  5 SNES Function norm 8.201793e-12
PETSc SNES solver converged in 5 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 45 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (3.9882284115387576, 2.10633905226147)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.163927e-01
  2 SNES Function norm 4.236739e-03
  3 SNES Function norm 5.531948e-06
  4 SNES Function norm 9.487613e-12
PETSc SNES solver converged in 4 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 46 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (1.8700681287445582, 9.444280175939497)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.359046e+00
  2 SNES Function norm 1.221518


=                        DEIM offline phase begins for                         =
=   { v_0 * (-1 + exp(f_9 * f_14)) / f_14 } * dx(<Mesh #0>[everywhere], {}),   =
=                                    where                                     =
=                                 f_14 = mu_1                                  =
=f_9 = solution of NonlinearEllipticDEIM (exact problem decorator: False, component: (None,))=
=               with id 49fc90be369b6ead57691b159f98bb7ecf8d1031               =

find initial mu
initial maximum interpolation error = 0.22755714686362255
initial maximum interpolation relative error = 1.0

:::::::::::::::::::::::::::::::::: DEIM N = 0 ::::::::::::::::::::::::::::::::::
solve interpolation for mu = (0.21630348466262952, 7.586202751823053)
compute and locate maximum interpolation error
update locations with (605,)
update basis
update interpolation matrix
interpolation error for current mu = 1.3877787807814457e-17
interpolation error on interpolation locatio

...

  4 SNES Function norm 8.924579e-01
  5 SNES Function norm 7.348105e-01
  6 SNES Function norm 1.688869e-01
  7 SNES Function norm 1.497795e-02
  8 SNES Function norm 1.384798e-04
  9 SNES Function norm 1.182400e-08
  10 SNES Function norm 3.746797e-15
PETSc SNES solver converged in 10 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 58 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (2.6016314178100957, 3.7443932479462885)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 3.490611e-01
  2 SNES Function norm 5.973206e-02
  3 SNES Function norm 1.937507e-03
  4 SNES Function norm 2.056199e-06
  5 SNES Function norm 2.331207e-12
PETSc SNES solver converged in 5 iterations with convergence reason 2.
add to snapshots

::::::::::::::::::::::::::::::::::: DEIM 59 ::::::::::::::::::::::::::::::::::::
evaluate parametrized expression at mu = (5.880120355611926, 2.7354908052204254)
  0 SNES Function norm 1.51

compute and locate maximum interpolation error
update locations with (128, 128)
update basis
update interpolation matrix
interpolation error for current mu = 6.505213034913027e-19
interpolation error on interpolation locations for current mu = 4.336808689942018e-19
find next mu
maximum interpolation error = 3.358020427399308e-05
maximum interpolation relative error = 1.8438705879042916e-05

::::::::::::::::::::::::::::::::: DEIM N = 11 ::::::::::::::::::::::::::::::::::
solve interpolation for mu = (1.4570291158403386, 4.885682243683056)
compute and locate maximum interpolation error
update locations with (367, 367)
update basis
update interpolation matrix
interpolation error for current mu = 1.3877787807814457e-17
interpolation error on interpolation locations for current mu = 6.938893903907228e-18
find next mu
maximum interpolation error = 3.060784649664067e-05
maximum interpolation relative error = 1.6806600535766824e-05

::::::::::::::::::::::::::::::::: DEIM N = 12 :::::::::::::::

  5 SNES Function norm 5.502969e-01
  6 SNES Function norm 8.742735e-02
  7 SNES Function norm 4.672271e-03
  8 SNES Function norm 1.456646e-05
  9 SNES Function norm 1.380663e-10
PETSc SNES solver converged in 9 iterations with convergence reason 2.
update snapshots matrix

###################################### 10 ######################################
truth solve for mu = (9.786397238905312, 7.993594056525069)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.358111e+00
  2 SNES Function norm 1.219347e+00
  3 SNES Function norm 1.094141e+00
  4 SNES Function norm 8.679181e-01
  5 SNES Function norm 5.921263e-01
  6 SNES Function norm 1.210197e-01
  7 SNES Function norm 8.002539e-03
  8 SNES Function norm 3.925863e-05
  9 SNES Function norm 9.331005e-10
PETSc SNES solver converged in 9 iterations with convergence reason 2.
update snapshots matrix

###################################### 11 ######################################
truth solve for mu = (4.620178828906789, 7.807

update snapshots matrix

###################################### 25 ######################################
truth solve for mu = (5.706265736474617, 4.39162911948858)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.146353e+00
  2 SNES Function norm 3.353682e-01
  3 SNES Function norm 4.967633e-02
  4 SNES Function norm 1.351914e-03
  5 SNES Function norm 1.055104e-06
  6 SNES Function norm 6.526607e-13
PETSc SNES solver converged in 6 iterations with convergence reason 2.
update snapshots matrix

###################################### 26 ######################################
truth solve for mu = (9.88385464221167, 1.0294276593728005)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 8.173385e-02
  2 SNES Function norm 9.037861e-04
  3 SNES Function norm 1.597979e-07
  4 SNES Function norm 6.644502e-15
PETSc SNES solver converged in 4 iterations with convergence reason 2.
update snapshots matrix

###################################### 27 ############################

  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 4.409840e-02
  2 SNES Function norm 1.891298e-04
  3 SNES Function norm 5.234305e-09
  4 SNES Function norm 4.187953e-15
PETSc SNES solver converged in 4 iterations with convergence reason 2.
update snapshots matrix

###################################### 44 ######################################
truth solve for mu = (5.763705490606231, 9.293669013786378)
  0 SNES Function norm 1.510614e+00
  1 SNES Function norm 1.358243e+00
  2 SNES Function norm 1.219164e+00
  3 SNES Function norm 1.093324e+00
  4 SNES Function norm 9.804222e-01
  5 SNES Function norm 7.347164e-01
  6 SNES Function norm 3.612940e-01
  7 SNES Function norm 5.749795e-02
  8 SNES Function norm 2.074486e-03
  9 SNES Function norm 2.842415e-06
  10 SNES Function norm 5.212172e-12
PETSc SNES solver converged in 10 iterations with convergence reason 2.
update snapshots matrix

###################################### 45 ######################################
truth so

#### 4.5.2 Train PINN

Given a training set $X_{PINN} = (\boldsymbol{\mu}^{(1)}, \dots, \boldsymbol{\mu}^{(n)})$ of parameters for the PDE, we train a Physics-Informed Neural Network (PINN) $\operatorname{N}_W(\boldsymbol{\mu})$ dependent on the weights and biases $W$ of the network to minimize the loss function

$$L_{PINN}(X_{PINN}; W) = \frac1n \sum_{i=1}^n \left\|A(\boldsymbol{\mu^{(i)}}) \operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \boldsymbol{f}(\boldsymbol{\mu}^{(i)}) + \boldsymbol{c}(\boldsymbol{\mu}^{(i)})\right\|_2^2$$

over $W$, where for a given $\boldsymbol{\mu}$, $A(\boldsymbol{\mu})$ is the assembled matrix corresponding to the bilinear form $a$, $\boldsymbol{f}(\boldsymbol{\mu})$ is the assembled vector corresponding to the linear form $f$, and $\boldsymbol{c}(\boldsymbol{\mu})$ is a vector corresponding to the nonlinear form $c$.

In [ ]:
input_normalization_pinn = Normalization.MinMaxNormalization(input_normalization=True)
output_normalization_pinn = Normalization.MinMaxNormalization()

pinn_net  = NN.RONN("PINN", problem, reduction_method, n_hidden=2, n_neurons=40)
pinn_loss = Losses.PINN_Loss(pinn_net, output_normalization_pinn)
data      = RONNData.RONNDataLoader(pinn_net, validation_proportion=0.2, 
                                    num_without_snapshots=100)
optimizer = torch.optim.Adam(pinn_net.parameters(), lr=0.001)
scheduler = None

pinn_trainer = Training.PINNTrainer(
    pinn_net, data, pinn_loss, optimizer, scheduler,
    input_normalization_pinn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    pinn_net, data, pinn_trainer, optimizer
)

In [ ]:
start = time.time()
pinn_trainer.train()
end = time.time()
print(end - start)

In [ ]:
fig, ax = Training.plot_loss(pinn_trainer, pinn_net)

#### 4.5.3 Train PDNN

Given a training set $X_{PDNN} = ((\boldsymbol{\mu}^{(1)}, \operatorname{HF}(\boldsymbol{\mu}^{(1)})), \dots, (\boldsymbol{\mu}^{(n)}, \operatorname{HF}(\boldsymbol{\mu}^{(n)})))$ of parameter and high fidelity solution pairs for the PDE, we train a Projection-Driven Neural Network (PDNN) $\operatorname{N}_W(\boldsymbol{\mu})$ dependent on the weights and biases $W$ of the network to minimize the loss function
$$L_{PDNN}(X_{PDNN}; W) = \frac1n \sum_{i=1}^n \|\operatorname{N}_W(\boldsymbol{\mu}^{(i)}) - \tilde{\operatorname{HF}}(\boldsymbol{\mu}^{(i)})\|_2^2,$$
where for a given $\boldsymbol{\mu}$, $\tilde{\operatorname{HF}}(\boldsymbol{\mu})$ is the projection of $\operatorname{HF}(\boldsymbol{\mu})$ onto the reduced order solution space.

In [ ]:
input_normalization_pdnn = Normalization.MinMaxNormalization(input_normalization=True)
output_normalization_pdnn = Normalization.MinMaxNormalization()

pdnn_net  = NN.RONN("PDNN", problem, reduction_method, n_hidden=2, n_neurons=40)
pdnn_loss = Losses.PDNN_Loss(pdnn_net, output_normalization_pdnn)
data      = RONNData.RONNDataLoader(pdnn_net, validation_proportion=0.2)
optimizer = torch.optim.Adam(pdnn_net.parameters(), lr=0.001)
scheduler = None

pdnn_trainer = Training.PDNNTrainer(
    pdnn_net, data, pdnn_loss, optimizer, scheduler,
    input_normalization_pdnn, num_epochs=10000
)

loaded, starting_epoch = IO.initialize_parameters(
    pdnn_net, data, pdnn_trainer, optimizer
)

In [ ]:
pdnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(pdnn_trainer, pdnn_net)

#### 4.5.4 Train PRNN

We train a Physics-Reinforced Neural Network (PRNN) $N_W(\boldsymbol{\mu})$ dependnent on the weights and biases $W$ of the network to minimize the loss function

$$L_{PRNN}(X_{PINN}, X_{PDNN}; W) = L_{PINN}(X_{PINN}; W) + \omega L_{PDNN}(X_{PDNN}; W),$$

where $\omega$ is a scaling parameter which can be chosen freely.

In [ ]:
input_normalization_prnn = Normalization.StandardNormalization(input_normalization=True)
output_normalization_prnn = Normalization.StandardNormalization()

omega = 1.
prnn_net  = NN.RONN(f"PRNN_{omega}", problem, reduction_method, n_hidden=2, n_neurons=40)
prnn_loss = Losses.PRNN_Loss(prnn_net, output_normalization_prnn, omega=omega)
data      = RONNData.RONNDataLoader(prnn_net, validation_proportion=0.2,
                                    num_without_snapshots=100)
optimizer = torch.optim.Adam(prnn_net.parameters(), lr=0.001)

prnn_trainer = Training.PRNNTrainer(
    prnn_net, data, prnn_loss, optimizer,
    input_normalization_prnn, num_epochs=1000
)

loaded, starting_epoch = IO.initialize_parameters(
    prnn_net, data, prnn_trainer, optimizer
)

In [ ]:
prnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(prnn_trainer, prnn_net, separate=True)

### 4.6. Perform an error analysis

#### 4.6.1 Reduction Method Error Analysis

In [ ]:
reduction_method.initialize_testing_set(100, DEIM=60)
#reduction_method.error_analysis()

#### 4.6.2 PINN Error Analysis

In [ ]:
test_mu = torch.tensor(reduction_method.testing_set)

In [ ]:
errors = ErrorAnalysis.error_analysis_fixed_net(
    pinn_net, test_mu, input_normalization_pinn, output_normalization_pinn, relative=True
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    pinn_net, (5., 5.), input_normalization_pinn, output_normalization_pinn, colorbar=True
)

#### 4.6.3 PDNN Error Analysis

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    pdnn_net, test_mu, input_normalization_pdnn, output_normalization_pdnn, relative=True
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    pdnn_net, (5., 5.), input_normalization_pdnn, output_normalization_pdnn
)

#### 4.6.4 PRNN Error Analysis

In [ ]:
_ = ErrorAnalysis.error_analysis_fixed_net(
    prnn_net, test_mu, input_normalization_prnn, output_normalization_prnn, relative=False
)

In [ ]:
ErrorAnalysis.plot_solution_difference(
    prnn_net, (5., 5.), input_normalization_prnn, output_normalization_prnn
)

#### 4.6.5 Neural Network Error Comparison

In [ ]:
nets = dict()
nets["pinn_net"] = pinn_net
nets["pdnn_net"] = pdnn_net
nets["prnn_net"] = prnn_net

input_normalizations = dict()
input_normalizations["pinn_net"] = input_normalization_pinn
input_normalizations["pdnn_net"] = input_normalization_pdnn
input_normalizations["prnn_net"] = input_normalization_prnn

output_normalizations = dict()
output_normalizations["pinn_net"] = output_normalization_pinn
output_normalizations["pdnn_net"] = output_normalization_pdnn
output_normalizations["prnn_net"] = output_normalization_prnn

_ = ErrorAnalysis.error_analysis_by_network(
    nets, test_mu, input_normalizations, output_normalizations, relative=False
)

### 4.7. Perform a speedup analysis

In [ ]:
reduction_method.speedup_analysis()

In [ ]:
for Cj in reduced_problem.assemble_operator('c'):
    print(Cj.reshape(-1)[0].content.shape)